## Creating a Classification Model

In this exercise, you will implement a classification model that uses features of a flight to predict whether or not the flight will be delayed.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1577379476625_0005,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights. The data includes specific characteristics (or *features*) for each flight, as well as a column indicating how many minutes late or early the flight arrived.

You will load this data into a DataFrame and display it.

In [6]:
csv = spark.read.csv('wasb:///data/flights.csv', inferSchema=True, header=True)
csv.show(5)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 5 rows

### Prepare the Data
Most modeling begins with exhaustive exploration and preparation of the data. In this example, the data has been cleaned for you. You will simply select a subset of columns to use as *features* and create a Boolean *label* field named **Late** with the value **1** for flights that arrived 15 minutes or more after the scheduled arrival time, or **0** if the flight was early or on-time.

(Note that in a real scenario, you would perform additional tasks such as handling missing or duplicated data, scaling numeric columns, and using a process called *feature engineering* to create new features for your model).

In [7]:
data = csv.select("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", ((col("ArrDelay") > 15).cast("Int").alias("Late")))
data.show(10)

+----------+---------+---------------+-------------+--------+----+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|Late|
+----------+---------+---------------+-------------+--------+----+
|        19|        5|          11433|        13303|      -3|   0|
|        19|        5|          14869|        12478|       0|   0|
|        19|        5|          14057|        14869|      -4|   0|
|        19|        5|          15016|        11433|      28|   1|
|        19|        5|          11193|        12892|      -6|   0|
|        19|        5|          10397|        15016|      -1|   0|
|        19|        5|          15016|        10397|       0|   0|
|        19|        5|          10397|        14869|      15|   1|
|        19|        5|          10397|        10423|      33|   1|
|        19|        5|          11278|        10397|     323|   1|
+----------+---------+---------------+-------------+--------+----+
only showing top 10 rows

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [8]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print "Training Rows:", train_rows, " Testing Rows:", test_rows

Training Rows: 1891828  Testing Rows: 810390

### Prepare the Training Data
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **VectorAssembler** class to transform the feature columns into a vector, and then rename the **Late** column to **label**.

In [10]:
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(train).select(col("features"), col("Late").alias("label"))
training.show(10)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    1|
|[1.0,1.0,10140.0,...|    0|
+--------------------+-----+
only showing top 10 rows

### Train a Classification Model
Next, you need to train a classification model using the training data. To do this, create an instance of the classification algorithm you want to use and use its **fit** method to train a model based on the training DataFrame. In this exercise, you will use a *Logistic Regression* classification algorithm - though you can use the same technique for any of the classification algorithms supported in the spark.ml API.

In [11]:
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
model = lr.fit(training)
print "Model trained!"

Model trained!

### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **Late** column to **trueLabel**.

In [14]:
testing = assembler.transform(test).select(col("features"), col("Late").alias("trueLabel"))
testing.show(10)

+--------------------+---------+
|            features|trueLabel|
+--------------------+---------+
|[1.0,1.0,10140.0,...|        0|
|[1.0,1.0,10140.0,...|        0|
|[1.0,1.0,10140.0,...|        0|
|[1.0,1.0,10140.0,...|        1|
|[1.0,1.0,10140.0,...|        0|
|[1.0,1.0,10299.0,...|        0|
|[1.0,1.0,10299.0,...|        0|
|[1.0,1.0,10299.0,...|        0|
|[1.0,1.0,10299.0,...|        0|
|[1.0,1.0,10299.0,...|        0|
+--------------------+---------+
only showing top 10 rows

### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict delay status for flights where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted status to the actual status. 

In [18]:
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "probability", "trueLabel")
predicted.show(10)

+--------------------+----------+--------------------+---------+
|            features|prediction|         probability|trueLabel|
+--------------------+----------+--------------------+---------+
|[1.0,1.0,10140.0,...|       0.0|[0.84296391101208...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.84296391101208...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.82561014180351...|        0|
|[1.0,1.0,10140.0,...|       1.0|[3.89575406721176...|        1|
|[1.0,1.0,10140.0,...|       0.0|[0.83044492555995...|        0|
|[1.0,1.0,10299.0,...|       0.0|[0.83840224120786...|        0|
|[1.0,1.0,10299.0,...|       0.0|[0.83281865829446...|        0|
|[1.0,1.0,10299.0,...|       0.0|[0.84602921301378...|        0|
|[1.0,1.0,10299.0,...|       0.0|[0.84987359396390...|        0|
|[1.0,1.0,10299.0,...|       0.0|[0.82522549157338...|        0|
+--------------------+----------+--------------------+---------+
only showing top 10 rows

Looking at the result, the **prediction** column contains the predicted value for the label, and the **trueLabel** column contains the actual known value from the testing data. It looks like there are a mix of correct and incorrect predictions - later in this course you'll learn how to measure the accuracy of a model.